## Main notebook to run Attention-LSTM models: Single Fold

Author: Lin Lee Cheong <br>
Last Updated: 11/23/2020 <br>

In [1]:
import os
import argparse
import time
import pickle
import pandas as pd
import numpy as np
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchtext.datasets import text_classification
from torchtext.vocab import Vocab
from attn_lstm_model import AttentionRNN
from model_utils import (
    log,
    build_lstm_dataset,
    epoch_train_lstm,
    epoch_val_lstm,
    generate_batch,
    count_parameters,
    epoch_time
)

**OPTIONS**

In [2]:
nrows = 1e9
min_freq = 500
device_id = None

train_data_path = "../../../data/readmission/fold_0/train/whole.csv"
valid_data_path = "../../../data/readmission/fold_0/test/whole_test.csv"
model_save_path = './lstm_model_30days/gen_attn_lstm_tmp'
results_path = './lstm_results_30days/gen_attn_lstm_tmp_results'

batch_size = 2046 #2046
N_EPOCHS = 5 #args.n_epochs

EMBEDDING_DIM = 30
HIDDEN_DIM = 30
BIDIRECTIONAL = False
DROPOUT = 0.3 # TODO: remove dropout

In [3]:
torch.cuda.is_available()
if device_id is None:
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
else:
    DEVICE = torch.device(f'cuda:{device_id}' if torch.cuda.is_available() else 'cpu')

In [4]:
for fp in [model_save_path, results_path]:
    if not os.path.isdir(os.path.split(fp)[0]):
        print(f'New directory created: {fp}')
        os.makedirs(os.path.split(fp)[0])

**READ IN TO GENERATE DATASET**

In [5]:
train_dataset = build_lstm_dataset(
    datapath=train_data_path, min_freq=500, nrows=nrows, rev=False
)

valid_dataset = build_lstm_dataset(
    datapath=valid_data_path,
    min_freq=500,
    nrows=nrows,
    vocab=train_dataset._vocab,
    rev=False,
)

log('vocab length:', len(train_dataset._vocab))

    0.00: Build token list
    7.85: Build counter
    8.61: Build vocab
    8.61: Build data
   10.07: Build pytorch dataset
   10.07: Skipped 0 invalid patients
   10.07: Skipped 44492 dead patients
   10.07: Done
   10.22: Build token list
   12.18: Build data
   12.49: Build pytorch dataset
   12.49: Skipped 0 invalid patients
   12.49: Skipped 11077 dead patients
   12.49: Done
   12.51: vocab length: 8935


In [6]:
# TODO: build LSTM dataset to use a provided vocabulary to process

In [7]:
# TODO: SAVE dataset, vocab
# torch.save(train_dataset, './tmp_train_dataset.pt')
# torch.save(valid_dataset,'./tmp_valid_datset.pt')

In [8]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=generate_batch,
    num_workers=8,
)
valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=generate_batch,
    num_workers=8
)

**MODEL GENERATION**

In [9]:
import torch
log(torch.cuda.is_available())
log(DEVICE)

   12.51: True
   12.51: cuda


In [10]:
INPUT_DIM = len(train_dataset._vocab) 
OUTPUT_DIM = len(train_dataset._labels)

In [11]:
model = AttentionRNN(       
    INPUT_DIM, 
    EMBEDDING_DIM, 
    HIDDEN_DIM, 
    OUTPUT_DIM, 
    BIDIRECTIONAL, 
    DROPOUT,
    padding_idx=0,
    device=DEVICE
)

model =  model.to(DEVICE)

log(model)
log(f'Nb of params: {count_parameters(model)}')

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


   12.56: AttentionRNN(
  (embedding): Embedding(8935, 30, padding_idx=0)
  (rnn): LSTM(30, 30, dropout=0.3)
  (fc): Linear(in_features=30, out_features=1, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)
   12.56: Nb of params: 275521


**MODEL TRAINING**

In [12]:
optimizer = optim.Adam(model.parameters(), lr=0.02)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 4, gamma=0.9)

#    optimizer = optim.SGD(model.parameters(), lr=args.lr)
#    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, gamma=0.9) #LLC-2/12: less aggresive drops
    
criterion = nn.BCEWithLogitsLoss() 
criterion = criterion.to(DEVICE)

In [13]:
log('Train')
best_valid_loss = float("inf")
valid_worse_loss = 0  # enable early stopping
stop_num = 6

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_auc = epoch_train_lstm(
        model, train_dataloader, optimizer, criterion
    )

    valid_loss, valid_auc = epoch_val_lstm(
        model, valid_dataloader, criterion, return_preds=False
    )

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f"Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), model_save_path)
        print("Saved Model, epoch {}".format(epoch))
        valid_worse_loss = 0

    else:
        valid_worse_loss += 1
        if valid_worse_loss == stop_num:
            print("EARLY STOP ------")
            break

    scheduler.step()
    log(
        f"Train Loss: {train_loss:.3f} | Train AUC: {train_auc:.2f} \t Val. Loss: {valid_loss:.3f} |  Val. AUC: {valid_auc:.4f}"
    )

   12.56: Train
Epoch: 01 | Epoch Time: 7m 52s
Saved Model, epoch 0
   20.44: Train Loss: 0.396 | Train AUC: 0.65 	 Val. Loss: 0.772 |  Val. AUC: 0.6796
Epoch: 02 | Epoch Time: 7m 59s
Saved Model, epoch 1
   28.43: Train Loss: 0.385 | Train AUC: 0.68 	 Val. Loss: 0.769 |  Val. AUC: 0.6821
Epoch: 03 | Epoch Time: 7m 58s
Saved Model, epoch 2
   36.40: Train Loss: 0.383 | Train AUC: 0.69 	 Val. Loss: 0.769 |  Val. AUC: 0.6821
Epoch: 04 | Epoch Time: 7m 57s
   44.37: Train Loss: 0.382 | Train AUC: 0.69 	 Val. Loss: 0.769 |  Val. AUC: 0.6819
Epoch: 05 | Epoch Time: 8m 0s
   52.38: Train Loss: 0.381 | Train AUC: 0.69 	 Val. Loss: 0.771 |  Val. AUC: 0.6823


## Get best model on val set: predictions, feature importance etc

In [14]:
# results = ( ids, predictions, labels, attn, events)
valid_loss, valid_auc, valid_results = epoch_val_lstm(
        model,
        valid_dataloader,
        criterion,
        return_preds=True
    )

In [15]:
torch.save(valid_results, results_path)